In [12]:
import pandas as pd
import numpy as np
import spacy
import time
from tqdm import tqdm
import gc

In [6]:
start_time = time.time()
train = pd.read_hdf('../input/train.h5')
test = pd.read_hdf('../input/test.h5')
train_text = train['comment_text']
test_text = test['comment_text']
text_list = pd.concat([train_text, test_text])
y = train['target'].values
num_train_data = y.shape[0]
print("--- %s seconds ---" % (time.time() - start_time))

--- 2.1337921619415283 seconds ---


In [11]:
# Third place solution
# seems like the kind of tokenizer you use doesn't matter too much
start_time = time.time()
print("Spacy NLP ...")
nlp = spacy.load('en_core_web_lg', disable=['parser','ner','tagger'])
nlp.vocab.add_flag(lambda s: s.lower() in spacy.lang.en.stop_words.STOP_WORDS, spacy.attrs.IS_STOP)
word_dict = {}
word_index = 1
lemma_dict = {}
docs = nlp.pipe(text_list, n_threads = 2)
word_sequences = []
for doc in tqdm(docs):
    word_seq = []
    for token in doc:
        if (token.text not in word_dict) and (token.pos_ is not "PUNCT"):
            word_dict[token.text] = word_index
            word_index += 1
            lemma_dict[token.text] = token.lemma_
        if token.pos_ is not "PUNCT":
            word_seq.append(word_dict[token.text])
    word_sequences.append(word_seq)
del docs
gc.collect()
train_word_sequences = word_sequences[:num_train_data]
test_word_sequences = word_sequences[num_train_data:]
print("--- %s seconds ---" % (time.time() - start_time))

Spacy NLP ...


1902194it [06:22, 4974.34it/s]


NameError: name 'gc' is not defined

In [16]:
test_word_sequences

[[2877,
  63652,
  2,
  804,
  90,
  74,
  1433,
  7,
  29476,
  2886,
  5,
  1325,
  7369,
  39,
  343,
  7420,
  1109,
  531,
  3651,
  93,
  4088,
  469,
  74,
  292,
  93,
  1434,
  7108,
  5],
 [61,
  367,
  7166,
  93,
  4982,
  56,
  516,
  14425,
  4206,
  16194,
  7,
  880,
  312526,
  3835,
  12983,
  45,
  1356,
  6537,
  39,
  3070,
  5,
  250,
  4890,
  6255,
  763,
  111,
  832,
  344,
  5005,
  45,
  6830,
  22627,
  2276,
  832,
  344,
  1963,
  86,
  93,
  925,
  1884,
  9,
  39,
  93,
  531,
  2589,
  101,
  13675,
  45,
  56133,
  3260,
  5,
  250,
  4079,
  93,
  28635,
  3771,
  832,
  404,
  8284,
  93,
  5528,
  1109,
  2589,
  101,
  4818,
  16,
  1215,
  5],
 [1052,
  37,
  265,
  41,
  5,
  426,
  7,
  606,
  32087,
  1975,
  56,
  4575,
  6376,
  5,
  528,
  899,
  93,
  443,
  20215,
  211,
  9,
  2941,
  7626,
  93,
  768,
  74,
  7637,
  466,
  6501,
  185,
  132,
  587,
  215,
  5146,
  5,
  250,
  61,
  4068,
  220,
  211,
  61,
  906,
  30,
  3513,
  85